In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import TaggedDocument
import string

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
data = pd.read_csv('newsCorpora-trimmed.csv')


In [ ]:
data.columns = ['category', 'text']

# Filtrar solo las categorías de interés
categories = ['b', 't', 'e', 'm']  # Business, Science and Technology, Entertainment, Health
data = data[data['category'].isin(categories)]

In [ ]:
# Preprocesamiento de texto
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

data['tokens'] = data['text'].apply(preprocess_text)


In [ ]:
# Crear documentos etiquetados
tagged_data = [TaggedDocument(words=row['tokens'], tags=[row['category']]) for index, row in data.iterrows()]

In [ ]:
from gensim.models import Doc2Vec

print(tagged_data)



In [ ]:
# Entrenar el modelo Doc2Vec
model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=5, workers=4, epochs=20)

# Guardar el modelo
model.save("doc2vec_model")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Convertir los documentos etiquetados a vectores
def vectorize_doc(doc):
    return model.infer_vector(doc.words)

data['vector'] = data['tokens'].apply(lambda x: vectorize_doc(TaggedDocument(x, [0])))


In [ ]:
# Crear el conjunto de entrenamiento y prueba
X = list(data['vector'])
y = data['category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Entrenar el clasificador SVM
classifier = SVC(kernel='linear')
classifier.fit(X_train, y_train)


In [ ]:
# Evaluar el clasificador
y_pred = classifier.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

In [ ]:
def classify_new_document(text):
    tokens = preprocess_text(text)
    vector = vectorize_doc(TaggedDocument(tokens, [0]))
    return classifier.predict([vector])[0]

# Ejemplo de clasificación
new_document = "New breakthrough in cancer research"
predicted_category = classify_new_document(new_document)
print(f"Predicted category: {predicted_category}")